In [1]:
import pandas as pd
import numpy as np
import altair as alt
from zipfile import ZipFile
import requests
import io


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
!pip install geopandas

In [138]:
import geopandas as gpd

In [16]:
#Source of data is https://ourworldindata.org/
df = pd.read_csv("electricty.csv")



In [4]:
#Renaming columns
df = df.rename(columns={'Access to electricity (% of population)':'% of people with electricity access', 'Entity': 'Country'})


In [5]:
#Removing non-countries
electricity2019 = df[df.Year==2019]

electricity2019 = electricity2019[electricity2019.Country !='Low income']

In [ ]:
#Finding out the bottom 20 countries
electricity2019 = electricity2019.sort_values(by='% of people with electricity access', ascending=False).tail(20)
electricity2019

In [7]:
#Plotting countries using altair

bars = alt.Chart(electricity2019, title="All 20 countries with lowest access to electricity are in Africa").mark_bar(size=15).encode(
    x='Country',
    y = alt.Y('% of people with electricity access', bin = False, scale=alt.Scale(domain=[0, 100])),
    opacity=alt.value(0.2),
    color=alt.value('blue')
).properties(width=400,
             height=250
).configure_axisX(labelAngle=40)


bars.save('chart.html', embed_options={'renderer':'svg'})

bars

alt.Chart(...)

In [8]:
# Importing Africa ISO Codes to filter data for Africa countries

iso = pd.read_csv("iso_a3.csv")
africa = iso.merge(df, on="Code")

In [9]:
#Creating a subset for 2019 alone and top countries in Africa
africa2019 = africa[africa.Year==2019]


africaTop = africa2019.sort_values(by='% of people with electricity access', ascending=False).head(20)

In [ ]:
africaTop

In [14]:
#Plotting countries in top 20 
bars2 = alt.Chart(africaTop, title="Only eight countries have electricity coverage of at least 90 percent").mark_bar(size=15).encode(
    x='Country_y',
    y = alt.Y('% of people with electricity access', bin = False, scale=alt.Scale(domain=[0, 100])),
    opacity=alt.value(0.2),
    color=alt.value('blue')
).properties(width=400,
             height=250
).configure_axisX(labelAngle=40)


bars2.save('africaTop.html')

In [15]:
#Loading countries file
countriesFile = gpd.read_file('countries.geojson')

NameError: ignored

In [ ]:
#Changing ISO_A3 to match Code column in CSV for merging
countriesFile.rename(columns={'ISO_A3':'Code'},inplace=True)

In [ ]:
mergedDF=electricity2019.merge(countriesFile, on='Code')


In [ ]:
mergedDF

,Entity,Code,Year,Access,ADMIN,geometry
0,Afghanistan,AFG,2019,97.700000,Afghanistan,"POLYGON ((71.04980 38.40866, 71.05714 38.40903..."
1,Albania,ALB,2019,100.000000,Albania,"POLYGON ((19.74777 42.57890, 19.74601 42.57993..."
2,Algeria,DZA,2019,99.500000,Algeria,"POLYGON ((8.60251 36.93951, 8.60566 36.91305, ..."
3,Andorra,AND,2019,100.000000,Andorra,"POLYGON ((1.70701 42.50278, 1.69750 42.49446, ..."
4,Angola,AGO,2019,45.670315,Angola,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
...,...,...,...,...,...,...
209,Venezuela,VEN,2019,100.000000,Venezuela,"MULTIPOLYGON (((-61.39027 8.58058, -61.36986 8..."
210,Vietnam,VNM,2019,99.400000,Vietnam,"MULTIPOLYGON (((106.66871 8.75349, 106.66586 8..."
211,Yemen,YEM,2019,72.751701,Yemen,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12..."
212,Zambia,ZMB,2019,43.000000,Zambia,"POLYGON ((31.11984 -8.61663, 31.14102 -8.60619..."


In [ ]:
#Converting to geojson
gdf = gpd.GeoDataFrame(mergedDF, crs="EPSG:4326", geometry=mergedDF.geometry)
gdf.to_file('Power2019.geojson', driver='GeoJSON')


In [ ]:
gdf.Access.min()

6.72053527832031